In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [9]:
value = """
def calibrate_null_class(boxes):
    sparse_classes, boxes = tf.split(boxes, [1, BOX_DIMS], axis=-1)
    sparse_classes += 1

    boxes = tf.concat([sparse_classes, boxes], axis=-1)
    return boxes

def dense_boxes(boxes):
    n_boxes = tf.shape(boxes)[0]
    empties = tf.zeros((MAX_BOX_QUERIES-n_boxes, BOX_DIMS+1), dtype=tf.float32)
    boxes = tf.concat([boxes, empties], axis=0)

    return boxes

dtrain_ds = train_ds.map(lambda x,y: (x, calibrate_null_class(y)))
dtrain_ds = dtrain_ds.map(lambda x,y: (x, dense_boxes(y)))
itr = iter(dtrain_ds)
y_true = next(itr)[1]
tf.print(y_true)
"""

key = 'dense_boxes'

print(shelf.get(key))
shelf[key] = value
shelf.sync()

None


## Enumerate

In [10]:
keys = list(shelf.keys())
print(len(keys), keys)

10 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment', 'run_summary_multioutput', 'show_dataset_samples', 'draw_bb_boxes', 'dense_boxes']


## Query

In [8]:
print(shelf['draw_bb_boxes'])


def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

def bbox_to_rect(bbox, shape, color):
    "Convert bounding box to matplotlib format."
    h, w = shape
    left, top, right, bottom = bbox[0]*w, bbox[1]*h, bbox[2]*w, bbox[3]*h
    
    rect = plt.Rectangle(
        xy=(left, top), width=right-left, height=bottom-top,
        fill=False, edgecolor=color, linewidth=2)
#     print(rect)
    return rect

N_CLASSES = ds_info.features['labels'].num_classes
LABELS = ds_info.features['labels'].names
CLASS_COLORS = list(map(lambda v: rgb_to_hex(tuple(v.tolist())), np.random.choice(range(64, 255),size=[N_CLASSES, 3])))

# itr = iter(train_ds)
itr = iter(raw_train_ds)
item = next(itr)

fig = plt.imshow(item['image'])

def draw_boxes(image, fig):
    box_shape = image.shape[:2]
    objs = item['objects']
    
    for index, bbox in enumerate(objs['bbox']):
        rect = bbox_to_rect(bbox, box_shape, CLASS_COLORS[objs['label'][index]])
        fig.axes.add_patch(rect)
        fig.axes.text(